In [20]:
import pandas as pd
import numpy as np
import plotly.express as px

# Importing and checking data

In [2]:
data_frame = pd.read_csv('data_orders.csv')

In [3]:
data_frame.head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,cancellations_time_in_seconds
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,4,1,198.0
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0
2,12:07:50,-0.969520,51.455544,477.0,3000582891479,4,1,46.0
3,13:50:20,-1.054671,51.460544,658.0,3000582941169,4,1,62.0
4,21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN


In [4]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_datetime                 10716 non-null  object 
 1   origin_longitude               10716 non-null  float64
 2   origin_latitude                10716 non-null  float64
 3   m_order_eta                    2814 non-null   float64
 4   order_gk                       10716 non-null  int64  
 5   order_status_key               10716 non-null  int64  
 6   is_driver_assigned_key         10716 non-null  int64  
 7   cancellations_time_in_seconds  7307 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 669.9+ KB


# Data cleaning

In [5]:
#dropping the columns that are irrelevant
data_frame2 = data_frame.drop(['origin_longitude', 'origin_latitude'], axis=1)

In [6]:
data_frame2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_datetime                 10716 non-null  object 
 1   m_order_eta                    2814 non-null   float64
 2   order_gk                       10716 non-null  int64  
 3   order_status_key               10716 non-null  int64  
 4   is_driver_assigned_key         10716 non-null  int64  
 5   cancellations_time_in_seconds  7307 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 502.4+ KB


In [7]:
#lets see how many missing values do we have in each column
data_frame2.isnull().sum()

order_datetime                      0
m_order_eta                      7902
order_gk                            0
order_status_key                    0
is_driver_assigned_key              0
cancellations_time_in_seconds    3409
dtype: int64

In [8]:
#extracting 'hours' from 'order_datetime' and dropping the 'order_datetime' column
data_frame2['hour'] = data_frame2['order_datetime'].str.split(':').apply(lambda split: split[0])
data_frame_cleaned = data_frame2.drop(['order_datetime'],axis=1)

In [9]:
#filling the missing values
data_frame_cleaned['cancellations_time_in_seconds'] = data_frame_cleaned['cancellations_time_in_seconds'].fillna(data_frame_cleaned.groupby(['hour','is_driver_assigned_key'])['cancellations_time_in_seconds'].transform('mean'))

In [10]:
#lets make colums "order_status_key" and "is_driver_assigned_key" more understandable for plotting
data_frame_cleaned["is_driver_assigned"] = np.where(data_frame_cleaned["is_driver_assigned_key"] == 1, "Yes", "No")
data_frame_cleaned["order_status"] = np.where(data_frame_cleaned["order_status_key"] == 4, "Client Cancelled", "System Reject")
data_frame_cleaned.drop(columns=["is_driver_assigned_key", "order_status_key"], inplace=True)


In [11]:
data_frame_cleaned.head()

,m_order_eta,order_gk,cancellations_time_in_seconds,hour,is_driver_assigned,order_status
0,60.0,3000583041974,198.000000,18,Yes,Client Cancelled
1,NaN,3000583116437,128.000000,20,No,Client Cancelled
2,477.0,3000582891479,46.000000,12,Yes,Client Cancelled
3,658.0,3000582941169,62.000000,13,Yes,Client Cancelled
4,NaN,3000583140877,113.959821,21,No,System Reject


In [12]:
#splitting data_frame_cleaned into two dataframes
columns = {'order_eta':data_frame_cleaned['m_order_eta'],'hour':data_frame_cleaned['hour']}
eta_data_frame = pd.DataFrame(data=columns)

In [13]:
eta_data_frame.head()

,order_eta,hour
0,60.0,18
1,NaN,20
2,477.0,12
3,658.0,13
4,NaN,21


In [14]:
#cleaning eta_data_frame
eta_data_frame_cleaned = eta_data_frame.dropna()
eta_data_frame_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2814 entries, 0 to 10715
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   order_eta  2814 non-null   float64
 1   hour       2814 non-null   object 
dtypes: float64(1), object(1)
memory usage: 66.0+ KB


# Data visualization

In [19]:
#creating new dataframe where we count the order_gk values by grouping the orders by order_status and is_driver_assigned
grouped_data_frame = data_frame_cleaned.groupby(['order_status', 'is_driver_assigned'])['order_gk'].count().reset_index()

#renaming the order_gk column
grouped_data_frame.rename(columns={'order_gk': 'order_count'}, inplace=True)

#plotting the distribution of orders according to reasons for failure
fig1 = px.histogram(grouped_data_frame,x='order_status',y='order_count', color='is_driver_assigned',barmode='group',
                   title='Distribution of Orders According to Reasons for Failure')
fig1.show()

We can conclude from the plot analysis that clients are more likely to cancel an order if a driver has not been assigned.

In [16]:
#creating new dataframe to count and group order_status and is_driver_assigned by hour
failed_by_order_status_and_driver_assignement = data_frame_cleaned.groupby(['hour', 'order_status','is_driver_assigned']).size().reset_index(name='count')

#merging columns order_status and is_driver_assigned into new column reasons
failed_by_order_status_and_driver_assignement['reasons'] = failed_by_order_status_and_driver_assignement['is_driver_assigned'] + ' - ' + failed_by_order_status_and_driver_assignement['order_status']

#renaming the column count
failed_by_order_status_and_driver_assignement.rename(columns={'count': 'number_of_failed_orders'}, inplace=True)

In [17]:
fig2 = px.line(failed_by_order_status_and_driver_assignement, x='hour', y='number_of_failed_orders', color='reasons',
              title='Distribution of Failed Orders by Hour')
fig2.show()

The highest numbers of cancelations occur at 21:00 (9 PM) and 08:00 (8 AM) also, during the night hours, clients cancellations with assigned driver are significantly lower.

In [18]:
#creating new dataframe by grouping hour and is_driver_assigned and calculating the mean(average) cancellations_time_in_seconds for the grouped data
avg_time_to_cancellation = data_frame_cleaned.groupby(['hour', 'is_driver_assigned']).agg({'cancellations_time_in_seconds': 'mean'}).reset_index()

#renaming cancellations_time_in_seconds column
avg_time_to_cancellation.rename(columns={'cancellations_time_in_seconds': 'avg_time_to_cancellation'}, inplace=True)

#plotting average time to cancellation by hour
fig3 = px.line(avg_time_to_cancellation, x='hour', y='avg_time_to_cancellation', color='is_driver_assigned', 
              title='Average Time to Cancellation by Hour (With and Without Driver)')
fig3.show()

During the early morning hours we can see that average time to cancellation has it's peak at 5:00AM when the driver is assigned, so I assume that clients cancelled the order because they have waited too long.

In [19]:
#creating new dataframe and calculating the average order_eta for each hour
avg_eta_by_hour = eta_data_frame_cleaned.groupby('hour').agg({'order_eta': 'mean'}).reset_index()

#renaming order_eta column
avg_eta_by_hour.rename(columns={'order_eta': 'avg_eta'}, inplace=True)

#creating a line plot
fig4 = px.line(avg_eta_by_hour, x='hour', y='avg_eta', 
              title='Distribution of Average ETA by Hour')
fig4.show()

The longest average ETA is at 8:00AM and this makes sense since the number of cancelled orders is really high around the same time.

# Solution to the problem

Add more drivers: Increasing the number of drivers can help meet the demand during peak hours, like 8:00 AM, when there are more orders and cancellations. This can potentially reduce waiting times and improve overall customer satisfaction.

Distribute drivers better by areas: Efficiently allocating drivers across different areas based on demand patterns can help ensure that there are enough drivers available in high-demand areas, which can lead to shorter waiting times and fewer cancellations.

Distribute drivers better by shifts: Optimizing driver shifts to match the demand patterns throughout the day can help ensure that there are enough drivers available during peak hours. By aligning driver availability with demand, you can potentially reduce waiting times and cancellations.